In [1]:
from ExpectationMaximization import ExpectationMaximization
import numpy as np
import matplotlib.pyplot as plt

In [2]:
kitti_path = '/scratch/local/hdd/hizkia/kitti'
date = None
reference_rectangle = (64, 128)
min_original_rectangle = (32, 64)

In [3]:
from scipy.stats import mode
def normalize_depth(depth, instance):
    return np.multiply(depth, instance == mode(instance.ravel())[0][0]) / np.median(depth[instance == mode(instance.ravel())[0][0]])

In [4]:
em = ExpectationMaximization(kitti_path,
                             date=date,
                             reference_rectangle=reference_rectangle,
                             min_original_rectangle=min_original_rectangle,
                             depth_normalization_func=normalize_depth,
                             max_iters = 5
                            )

In [ ]:
em.run()

Generating samples...
Error with /scratch/local/hdd/hizkia/kitti/2011_09_29/2011_09_29_drive_0026_sync/image_02/data/0000000134.jpg
Error with /scratch/local/hdd/hizkia/kitti/2011_09_30/2011_09_30_drive_0034_sync/image_02/data/0000000729.jpg
Error with /scratch/local/hdd/hizkia/kitti/2011_09_30/2011_09_30_drive_0034_sync/image_02/data/0000001011.jpg
Error with /scratch/local/hdd/hizkia/kitti/2011_09_30/2011_09_30_drive_0034_sync/image_02/data/0000001213.jpg
Initializing EM yields a score of 241041409.9207095...
Error with /scratch/local/hdd/hizkia/kitti/2011_09_29/2011_09_29_drive_0026_sync/image_02/data/0000000134.jpg
Error with /scratch/local/hdd/hizkia/kitti/2011_09_30/2011_09_30_drive_0034_sync/image_02/data/0000000729.jpg
Error with /scratch/local/hdd/hizkia/kitti/2011_09_30/2011_09_30_drive_0034_sync/image_02/data/0000001011.jpg
Error with /scratch/local/hdd/hizkia/kitti/2011_09_30/2011_09_30_drive_0034_sync/image_02/data/0000001213.jpg
